In [4]:
from usosapi import USOSAPIConnection
from webbrowser import open
from functools import reduce

# Logging in
Firstly, we are creating new connection using consumer key and secret. Then, user need to perform authorization via browser and insert given pin number.

In [5]:
connection = USOSAPIConnection(api_base_address='https://apps.usos.pwr.edu.pl/',
                               consumer_key='ssijmi',
                               consumer_secret='chuja')
connection_url = connection.get_authorization_url()
open(connection_url)
pin = input()

KeyError: 'Decoder failed to handle oauth_token with data as returned by provider. A different decoder may be needed. Provider returned: b\'{"message": "Invalid consumer."}\''

USOSapi creates connection and test it.

In [3]:
connection.authorize_with_pin(pin)
connection.is_authorized()

True

Now we are able to download logged-in user's registrations.

<div class="alert alert-block alert-warning">
<b>Warning:</b> We are not sure how will list behave when new registrations will appear. We may introduce choose of right registrations.
</div>

In [196]:
registration = connection.get('services/registrations/user_registrations')[0]

Downloading and picking courses' ids.

In [197]:
courses = [x['course_id'] for x in connection.get('services/registrations/user_registrations', id=registration['id'], fields='related_courses')[0]['related_courses']]

# EXAMPLE
Picking one course. I've chose Database design with Nguyen because the lecture is presented once every two weeks.

In [232]:
course_id = courses[9]
connection.get('services/courses/course', course_id=course_id)

{'id': 'W04IST-SI0815G',
 'name': {'pl': 'Projektowanie baz danych', 'en': 'Database Design'}}

Getting term to use new API method, *services/courses/course'*.

In [231]:
term = connection.get('services/courses/course', course_id=course_id, fields='terms')['terms'][0]['id']
term

'2023/24-Z'

In this term course's edition we have lecture and project classes.

In [214]:
units = connection.get('services/courses/course_edition2', course_id=course_id, term_id=term, fields='course_units')['course_units']
units

[{'id': '52863'}, {'id': '52864'}]

We pick the first one. Fun fact: we are able to see if this chose is right in the next steps, empirically, not yet.

In [224]:
unit = units[0]['id']

In [225]:
unit

'52863'

In [226]:
connection.get('services/courses/unit',unit_id=unit, )

{'id': '52863'}

Now, look at the unit's groups.

In [227]:
class_groups = connection.get('services/courses/course_unit',course_unit_id=unit, fields='class_groups')['class_groups']

In [228]:
class_groups

[{'course_unit_id': '52863', 'number': 1}]

Hurrah. If it has only one group, it is the lecture - we've chosen right unit.

In [233]:
connection.get('services/groups/class_group',course_unit_id=class_groups[0]['course_unit_id'],              group_number=class_groups[0]['number'], fields='lecturers')

{'lecturers': [{'id': '1309', 'first_name': 'Ngoc', 'last_name': 'Nguyen'}]}

Next fun fact: there is no **Than** between Ngoc and Nguyen, because programmers didn't create place for the second name. Bad business analysis?
And then we are able to look into dates of each of lectures.

In [235]:
connection.get('services/tt/classgroup_dates2',
               unit_id=class_groups[0]['course_unit_id'],
               group_number=class_groups[0]['number'], 
               fields='start_time|end_time|name|cgwm_id|classtype_id|unit_id')

[{'start_time': '2023-10-03 11:15:00',
  'end_time': '2023-10-03 12:00:00',
  'name': {'pl': 'Wykład', 'en': 'Lecture'},
  'cgwm_id': 26093,
  'classtype_id': 'W',
  'unit_id': 52863},
 {'start_time': '2023-10-10 11:15:00',
  'end_time': '2023-10-10 13:00:00',
  'name': {'pl': 'Wykład', 'en': 'Lecture'},
  'cgwm_id': 26093,
  'classtype_id': 'W',
  'unit_id': 52863},
 {'start_time': '2023-10-24 11:15:00',
  'end_time': '2023-10-24 13:00:00',
  'name': {'pl': 'Wykład', 'en': 'Lecture'},
  'cgwm_id': 26093,
  'classtype_id': 'W',
  'unit_id': 52863},
 {'start_time': '2023-11-07 11:15:00',
  'end_time': '2023-11-07 13:00:00',
  'name': {'pl': 'Wykład', 'en': 'Lecture'},
  'cgwm_id': 26093,
  'classtype_id': 'W',
  'unit_id': 52863},
 {'start_time': '2023-11-21 11:15:00',
  'end_time': '2023-11-21 13:00:00',
  'name': {'pl': 'Wykład', 'en': 'Lecture'},
  'cgwm_id': 26093,
  'classtype_id': 'W',
  'unit_id': 52863},
 {'start_time': '2023-12-05 11:15:00',
  'end_time': '2023-12-05 13:00:00',

# Question
##### What do we do now?
I did not find any simpler methods to extract data about odd/even weeks and day of the week. Therefore, if there is no such method, we must derive necessary data from lists such as this one above, with start_time and end_time and only these. It is not very hard task, nevertheless **how** to conduct such translation of information is open. Buzi.

In addition: we may use special methods to request for many entities at once.

In [240]:
courses[:3]

['W04IST-SI0011L', 'W04IST-SI0011W', 'W04IST-SI0021L']

In [242]:
course_ids = reduce(lambda x, y: x + '|' + y, courses)
connection.get('services/courses/courses', course_ids=course_ids)

{'W04IST-SI0011L': {'id': 'W04IST-SI0011L',
  'name': {'pl': 'Podstawy internetu rzeczy', 'en': 'Introduction to IoT'}},
 'W04IST-SI0011W': {'id': 'W04IST-SI0011W',
  'name': {'pl': 'Podstawy internetu rzeczy', 'en': 'Introduction to IoT'}},
 'W04IST-SI0021L': {'id': 'W04IST-SI0021L',
  'name': {'pl': 'Cyberbezpieczeństwo', 'en': 'Cybersecurity'}},
 'W04IST-SI0021W': {'id': 'W04IST-SI0021W',
  'name': {'pl': 'Cyberbezpieczeństwo', 'en': 'Cybersecurity'}},
 'W04IST-SI0022P': {'id': 'W04IST-SI0022P',
  'name': {'pl': 'Projektowanie oprogramowania',
   'en': 'Software Engineering'}},
 'W04IST-SI0022W': {'id': 'W04IST-SI0022W',
  'name': {'pl': 'Projektowanie oprogramowania',
   'en': 'Software Engineering'}},
 'W04IST-SI0811G': {'id': 'W04IST-SI0811G',
  'name': {'pl': 'Programowanie systemów webowych',
   'en': 'Web Systems Programming'}},
 'W04IST-SI0812G': {'id': 'W04IST-SI0812G',
  'name': {'pl': 'Aplikacje webowe na platformę .NET',
   'en': '.NET Web Applications'}},
 'W04IST-SI0813